In [1]:
import os
os.chdir("..")

In [2]:
import hydra
from omegaconf import DictConfig

from shell.fleet.helper import get_fleet, get_agent_cls
import torch
import time
import datetime
import logging
from shell.utils.experiment_utils import setup_experiment, process_dataset_cfg, eval_net
from shell.datasets.datasets import get_dataset
from pprint import pprint
import ray
import os
from hydra import compose, initialize
from omegaconf import OmegaConf
from shell.utils.utils import seed_everything
from shell.fleet.model_sharing_utils import is_in, diff_models
logging.basicConfig(level=logging.INFO)

In [3]:
initialize(config_path="conf", job_name="tmp_job")
cfg = compose(config_name="grad_mod")
seed_everything(cfg.seed)

/tmp/ipykernel_552631/3645537571.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="conf", job_name="tmp_job")
/home/vlongle/miniconda3/envs/shell/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'grad_mod': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
# override
cfg.parallel = True
cfg.train.num_epochs = 50
cfg.train.component_update_freq = 50


# cfg.train.num_epochs = 1
# cfg.train.component_update_freq = 1

In [5]:
AgentCls = get_agent_cls(cfg.sharing_strategy, cfg.algo, cfg.parallel)

graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg = setup_experiment(
        cfg)
FleetCls = get_fleet(cfg.sharing_strategy, cfg.parallel)

INFO:root:Class sequence: [4 7 0 8 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)


{'train': {'component_update_freq': 50, 'num_epochs': 50, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'gradient', 'num_coms_per_round': 5, 'retrain': {'num_epochs': 10}}, 'seed': 0, 'algo': 'modular', 'job_name': 'fun', 'num_agents': 4, 'root_save_dir': 'results', 'parallel': True, 'num_init_tasks': 2, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 64, 'improvement_threshold': 0.05, 'use_contrastive': True}}


INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [2 1 3 8 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 9 7 2 1 8 6 1 6 4 5 7 8 0 2 3 0 3 9 6]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 5 3 4 4 5 9 3 0 6 9 1 3 1 7 9 5 2 3 2]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(12

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 2}
<class 'shell.learners.er_dynamic.CompositionalDynamicER'>


In [6]:
fake_dataset = get_dataset(**process_dataset_cfg(cfg))
fleet = FleetCls(graph, cfg.seed, datasets, cfg.sharing_strategy, AgentCls, NetCls=NetCls,
                     LearnerCls=LearnerCls, net_kwargs=net_cfg, agent_kwargs=agent_cfg,
                     train_kwargs=train_cfg, fake_dataset=fake_dataset)

INFO:root:Class sequence: [3 9 2 4 9 2 5 4 8 9 9 4 1 6 9 6 8 0 5 1]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
2023-03-28 18:18:51,910	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(ParallelModGrad pid=560813) INFO:root:Agent: node_id: 69420, seed: 69420000
(ParallelModGrad pid=560813) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=560813) INFO:root:	task: 0	loss: 6.208	acc: 0.500
(ParallelModGrad pid=560813) INFO:root:	task: avg	loss: 6.208	acc: 0.500
(ParallelModGrad pid=560813) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=560813) INFO:root:	task: 0	loss: 6.201	acc: 0.500
(ParallelModGrad pid=560813) INFO:root:	task: avg	loss: 6.201	acc: 0.500
(ParallelModGrad pid=560813) INFO:root:final com

In [7]:
print("init tasks:", fake_dataset.class_sequence)
print("agent tasks")
pprint([data.class_sequence for data in datasets])

init tasks: [3 9 2 4 9 2 5 4 8 9 9 4 1 6 9 6 8 0 5 1]
agent tasks
[array([4, 7, 0, 8, 4, 6, 4, 3, 8, 0, 1, 0, 5, 9, 6, 4, 9, 0, 3, 0]),
 array([2, 1, 3, 8, 6, 0, 3, 0, 5, 0, 3, 6, 2, 7, 6, 7, 6, 1, 0, 5]),
 array([0, 9, 7, 2, 1, 8, 6, 1, 6, 4, 5, 7, 8, 0, 2, 3, 0, 3, 9, 6]),
 array([0, 5, 3, 4, 4, 5, 9, 3, 0, 6, 9, 1, 3, 1, 7, 9, 5, 2, 3, 2])]


In [8]:
print("After initialization:")
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

After initialization:


{'components.0.weight': 0.0,
 'components.0.bias': 0.0,
 'components.1.weight': 0.0,
 'components.1.bias': 0.0,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [9]:
print("Accuracy just after initialization:")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

Accuracy just after initialization:
{0: 0.4900497512437811}
{0: 0.5237655745269958}
{0: 0.4927099044746104}
{0: 0.4754273504273504}


In [10]:
# local train
fleet.train(0)

(ParallelModGrad pid=620538) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=620538) INFO:root:	task: 0	loss: 6.188	acc: 0.490
(ParallelModGrad pid=620538) INFO:root:	task: avg	loss: 6.188	acc: 0.490
(ParallelModGrad pid=620586) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=620586) INFO:root:	task: 0	loss: 6.237	acc: 0.524
(ParallelModGrad pid=620586) INFO:root:	task: avg	loss: 6.237	acc: 0.524
(ParallelModGrad pid=620634) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=620634) INFO:root:	task: 0	loss: 6.145	acc: 0.493
(ParallelModGrad pid=620634) INFO:root:	task: avg	loss: 6.145	acc: 0.493
(ParallelModGrad pid=620693) INFO:root:epochs: 0, training task: 0
(ParallelModGrad pid=620693) INFO:root:	task: 0	loss: 6.331	acc: 0.475
(ParallelModGrad pid=620693) INFO:root:	task: avg	loss: 6.331	acc: 0.475
(ParallelModGrad pid=620693) INFO:root:epochs: 1, training task: 0
(ParallelModGrad pid=620693) INFO:root:	task: 0	loss: 6.207	acc: 0.606
(ParallelModGrad p

In [11]:
print("After local training:")

diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

After local training:


{'components.0.weight': 0.0028489402029663324,
 'components.0.bias': 0.0028158421628177166,
 'components.1.weight': 0.0027418900281190872,
 'components.1.bias': 0.003049996914342046,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [12]:
print("Accuracy after local train:")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

Accuracy after local train:
{0: 0.9238805970149254}
{0: 0.9510844485463775}
{0: 0.971342383107089}
{0: 0.75}


In [13]:
# check the number of components
for agent in fleet.agents:
    print(len(ray.get(agent.get_net.remote()).components))

2
3
2
3


TODO: check that structures, decoders, encoders and non-shared modules are NOT changed because they are not being
aggregated and also not being finetuned.

In [14]:
net1_local = ray.get(fleet.agents[0].get_net.remote()) # net1 after local training.

In [15]:
fleet.num_coms_per_round = 10
fleet.communicate(0)

In [16]:
print("Accuracy after sync:")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

Accuracy after sync:
{0: 0.927860696517413}
{0: 0.9478541762805722}
{0: 0.9703368526897939}
{0: 0.8637820512820513}


In [17]:
print("After sync between agents:")

diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

After sync between agents:


{'components.0.weight': 0.0019303187727928162,
 'components.0.bias': 0.0022637085057795048,
 'components.1.weight': 0.0018625843804329634,
 'components.1.bias': 0.0022147209383547306,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [18]:
print("After sync between agent1 (local) and agent1 (sync):")
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            net1_local.state_dict(),)

After sync between agent1 (local) and agent1 (sync):


{'structure.0': 0.0,
 'structure.1': 0.0,
 'structure.2': 0.0,
 'structure.3': 0.0,
 'structure.4': 0.0,
 'structure.5': 0.0,
 'structure.6': 0.0,
 'structure.7': 0.0,
 'structure.8': 0.0,
 'structure.9': 0.0,
 'components.0.weight': 0.0056788865476846695,
 'components.0.bias': 0.005201797932386398,
 'components.1.weight': 0.005264018662273884,
 'components.1.bias': 0.00524795800447464,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0,
 'decoder.0.weight': 0.019980832934379578,
 'decoder.0.bias': 0.011504922062158585,
 'decoder.1.weight': 0.0,
 'decoder.1.bias': 0.0,
 'decoder.2.weight': 0.0,
 'decoder.2.bias': 0.0,
 'decoder.3.weight': 0.0,
 'decoder.3.bias': 0.0,
 'decoder.4.weight': 0.0,
 'decoder.4.bias': 0.0,
 'decoder.5.weight': 0.0,
 'decoder.5.bias': 0.0,
 'decoder.6.weight': 0.0,
 'decoder.6.bias': 0.0,
 'decoder.7.weight': 0.0,
 'decoder.7.bias': 0.0,
 'decoder.8.weight': 0.0,
 'decoder.8.bias': 0.0,
 'decoder.9.weight': 0.0,
 'decoder.9.bias': 0

(raylet) [2023-03-29 01:12:52,282 E 555291 555291] (raylet) node_manager.cc:3040: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: d05aa95ca8792bd0128332767b10b7bec22cda1c8aa3fef916ab42b8, IP: 158.130.50.18) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 158.130.50.18`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-03-29 15:05:53,064 E 555291 555291] (raylet) node_manager.cc:3040: 1 Workers (tasks / actors) kill